In [38]:
import scraping_class
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [39]:
def get_awards(actorlist):
    
    awards = 'awards.txt'
    connector = scraping_class.Connector(awards)
    
    nom = 0
    win = 0
    
    for i in actorlist:
        url = str(i)
        response, call_id = connector.get(url, 'awards')

        html = response.text
        soup = BeautifulSoup(html,'html.parser')

        table_node = soup.find_all('table', attrs ={'class': 'awards'})

        awards_data = []

        for i in table_node:
            try:
                award_year = i.find_all('td', attrs ={'class': 'award_year'})
                award_year =[i.find('a').text.strip('\n') for i in award_year]
                outcome = i.find_all('td', attrs = {'class': 'award_outcome'})

                outcome = [i.text for i in outcome]

                award = [i.split('\n')[2] for i in outcome]
                result = [i.split('\n')[1] for i in outcome]

                if award[0] == 'Oscar':
                    awards_data.append([award_year, award, result])
            except:
                NameError

        df = pd.DataFrame(awards_data[0]).T

        df = df.assign(nom = lambda df: pd.Series.str(df[2])[0:] == 'Nominee')
        df = df.assign(win = lambda df: pd.Series.str(df[2])[0:] == 'Winner')


        for i in df.nom:
            if i == True:
                nom += 1

        for i in df.win:
            if i == True:
                win += 1

    return nom, win